In [1]:
#https://github.com/adventuresinML/adventures-in-ml-code/blob/master/keras_lstm.py
    
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from pandas import read_csv
from pandas import DataFrame
from pandas import concat

# for scaling and inverse_transform
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import load_model
 
import pickle

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1): # range([start,] stop [, step])
        # DataFrame.shift(periods=1, freq=None, axis=0), returns DataFrame
		cols.append(df.shift(i)) 
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
    #  concatenating along the columns (axis=1), a DataFrame is returned.
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.


In [4]:
# load dataset
dataset = read_csv('bb-cross-train.csv', header=0)
values = dataset.values
print(values.shape)

# ensure all data is float
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
print(scaler.data_max_)
print(scaler.data_min_)

pickle.dump(scaler, open("min-max-scaler.pkl", 'wb'))

# specify the number of lag hours
n_seq = 3
n_features = 4
# frame as supervised learning
reframed = series_to_supervised(scaled, n_seq, 1)
print(reframed.shape)

# split into train and test sets
values = reframed.values
n_train_num = 84372 - 20000
train = values[:n_train_num, :]
val = values[n_train_num:, :]

# split into input and outputs
n_obs = n_seq * n_features
train_X, train_y = train[:, :n_obs], train[:, n_obs:n_obs+n_features]
val_X, val_y = val[:, :n_obs], val[:, n_obs:n_obs+n_features]
print(train_X.shape, len(train_X), train_y.shape)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_seq, n_features))
val_X = val_X.reshape((val_X.shape[0], n_seq, n_features))
print(train_X.shape, train_y.shape, val_X.shape, val_y.shape)


(84371, 4)
[1900.  953. 1919. 1079.]
[0. 0. 0. 0.]
(84368, 16)
(50000, 12) 50000 (50000, 4)
(50000, 3, 4) (50000, 4) (34368, 3, 4) (34368, 4)


In [170]:
scaled[0]

array([0.87315786, 0.6054564 , 0.9484106 , 0.90546805], dtype=float32)

In [161]:
import pickle
example_dict = {1:"6",2:"2",3:"f"}

pickle.dump(example_dict, open("dict.pickle","wb"))

In [131]:
train_X.shape[2]

4

In [132]:
val_X.shape

(49846, 3, 4)

In [26]:
 # design network
K.clear_session()  # Clear previous models from memory.    
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(4))
model.compile(loss='mae', optimizer='adam')

epochs = 5000
# fit network
history = model.fit(train_X, train_y, epochs=epochs, batch_size=72, validation_data=(val_X, val_y), verbose=2, shuffle=False)

# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

# save the model
model.save(conf1-lstm-model.h5)

Train on 50000 samples, validate on 34368 samples
Epoch 1/5000


InternalError: GPU sync failed

In [2]:
model = load_model("jaad-bb-lstm-model.h5")
# make a prediction
yhat = model.predict(test_X)

# invert scaling for forecast
inv_yhat = scaler.inverse_transform(yhat)

# invert scaling for actual
inv_y = scaler.inverse_transform(test_y)

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

NameError: name 'test_X' is not defined

In [134]:
(train_X).shape

(50000, 3, 4)

In [135]:
val_X.shape

(49846, 3, 4)

In [136]:
( data.values.shape)

(99849, 4)

In [137]:
print(val_X[0])

[[0.59631574 0.73137456 0.6388744  0.79332715]
 [0.5947368  0.7303252  0.6373111  0.79425395]
 [0.5926316  0.7282266  0.63522667 0.79518074]]


In [138]:
n_features

4

In [139]:
val_y.shape

(49846, 4)

In [ ]:
# load dataset
dataset = read_csv('bb-cross-train.csv', header=0)
values = dataset.values
print(values.shape)

# ensure all data is float
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
print(scaler.data_max_)
print(scaler.data_min_)

pickle.dump(scaler, open("min-max-scaler.pkl", 'wb'))

# specify the number of lag hours
n_seq = 3
n_features = 4
# frame as supervised learning
reframed = series_to_supervised(scaled, n_seq, 1)
print(reframed.shape)

# split into train and test sets
values = reframed.values
n_train_num = 50000
train = values[:n_train_num, :]
val = values[n_train_num:, :]

# split into input and outputs
n_obs = n_seq * n_features
train_X, train_y = train[:, :n_obs], train[:, n_obs:n_obs+n_features]
val_X, val_y = val[:, :n_obs], val[:, n_obs:n_obs+n_features]
print(train_X.shape, len(train_X), train_y.shape)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_seq, n_features))
val_X = val_X.reshape((val_X.shape[0], n_seq, n_features))
print(train_X.shape, train_y.shape, val_X.shape, val_y.shape)


In [13]:
# load test dataset
values = read_csv('bb-cross-test1.csv', header=0).values
print(len(values))
# ensure all data is float
values = values.astype('float32')

# normalize features

scaled = scaler.transform(values)
print(scaler.data_max_)
print(scaler.data_min_)

# specify the number of lag hours
n_seq = 3
n_features = 4
# frame as supervised learning
reframed = series_to_supervised(scaled, n_seq, 1)
print(reframed.shape)
values = reframed.values

# split into input and outputs
n_obs = n_seq * n_features
test_X, test_y = values[:, :n_obs], values[:, n_obs:n_obs+n_features]
print(test_X.shape, len(test_X), test_y.shape)

# reshape input to be 3D [samples, timesteps, features]
test_X = test_X.reshape((test_X.shape[0], n_seq, n_features))
print(test_X.shape, test_y.shape)

204
[1900.  953. 1919. 1079.]
[0. 0. 0. 0.]
(201, 16)
(201, 12) 201 (201, 4)
(201, 3, 4) (201, 4)


In [14]:
K.clear_session()  # Clear previous models from memory.    
model = load_model("jaad-bb-lstm-model.h5")
# make a prediction
yhat = model.predict(test_X)

# invert scaling for forecast
inv_yhat = scaler.inverse_transform(yhat)

# invert scaling for actual
inv_y = scaler.inverse_transform(test_y)

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 2.038


In [124]:
test_X.shape

(49846, 12)

In [143]:
test_X[0]

array([[0.59631574, 0.73137456, 0.6388744 , 0.79332715],
       [0.5947368 , 0.7303252 , 0.6373111 , 0.79425395],
       [0.5926316 , 0.7282266 , 0.63522667, 0.79518074]], dtype=float32)

In [144]:
yhat.shape

(49846, 4)

In [151]:
import numpy as np
yhat =  model.predict(np.expand_dims(test_X[0], axis=0))

In [152]:
inv_yhat = scaler.inverse_transform(yhat)

In [153]:
inv_yhat

array([[1122.4075,  692.9786, 1214.8446,  859.0956]], dtype=float32)